# Fake currency detection using ResNet50 model

In [1]:
#import all necessary modules
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array 
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#define height and width of the image
height=300
width=300

#create a ResNet50 model instance without the top layer as we will add our own top layer
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(height,width,3))

In [3]:
#define directory containing training and validation data
train_dir="dataset/training"
validation_dir="dataset/validation"

#number of batches the data has to be divided into
batch_size=8

#create datagen and generator to load the data from training directory
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(height,width),batch_size=batch_size)

#create datagen and generator to load the data from validation directory
validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=90,horizontal_flip=True,vertical_flip=True)
validation_generator=validation_datagen.flow_from_directory(validation_dir,target_size=(height,width),batch_size=batch_size)

Found 2774 images belonging to 2 classes.
Found 592 images belonging to 2 classes.


In [4]:
def build_finetune_model(base_model,dropout,fc_layers,num_classes):
    for layer in base_model.layers:
        layer.trainable=False

    x=base_model.output
    x=Flatten()(x)
    for fc in fc_layers:
        x=Dense(fc,activation='relu')(x)
        x=Dropout(dropout)(x)
    
    predictions=Dense(num_classes,activation='softmax')(x)

    finetune_model=Model(inputs=base_model.input,outputs=predictions) 
    
    return finetune_model

class_list=['Real','Fake'] #the labels of our data
FC_Layers=[1024,1024]
dropout=0.5

finetune_model=build_finetune_model(base_model,dropout=dropout,fc_layers=FC_Layers,num_classes=len(class_list))

In [5]:
num_epochs=20
num_train_images=2774

In [6]:

#checkpoint in case anything goes wrong
checkpoint=ModelCheckpoint("Final_model.h5",monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
early=EarlyStopping(monitor='val_accuracy',min_delta=0,patience=40,verbose=1,mode="auto")

#compile the model before using
optimizer = optimizers.SGD(learning_rate=0.001, momentum=0.9)
finetune_model.compile(loss="categorical_crossentropy",optimizer=optimizer, metrics=['accuracy'])

#train the model
finetune_model.fit(train_generator,steps_per_epoch=num_train_images//batch_size,epochs=num_epochs,validation_data=validation_generator,validation_steps=1,callbacks=[checkpoint,early])

#save the model
finetune_model.save_weights("Final_model.h5")

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4954 - loss: 21954668078825472.0000

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from None to 0.75000, saving model to Final_model.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 796s 2s/step - accuracy: 0.5011 - loss: 45607269873745920.0000 - val_accuracy: 0.7500 - val_loss: 415.2369
Epoch 2/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 17:38 3s/step - accuracy: 0.2500 - loss: 1278.3269

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.2500 - loss: 1278.3269 - val_accuracy: 0.3750 - val_loss: 1058.2571
Epoch 3/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.4973 - loss: 4000011255808.0000
Epoch 3: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 2624s 8s/step - accuracy: 0.4906 - loss: 1232771743744.0000 - val_accuracy: 0.5000 - val_loss: 0.6944
Epoch 4/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 16:10 3s/step - accuracy: 0.3750 - loss: 0.7071
Epoch 4: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.3750 - loss: 0.7071 - val_accuracy: 0.2500 - val_loss: 0.7193
Epoch 5/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4952 - loss: 4599066.0000
Epoch 5: val_accuracy did not improve from 0.75000
346/346 ━━━━━━━━━━━━━━━━━━━━ 746s 2s/step - accuracy: 0.4881 - loss: 50774976.0000 - val_accuracy: 0.6250 - val_loss: 0.6695
Epoch 6/20
  1/346 ━━━━

ValueError: The filename must end in `.weights.h5`. Received: filepath=Final_model.h5